# Загрузка библиотек. Подготовка данных

In [6]:
import csv
import os
from itertools import combinations
import glob
import pandas as pd

In [4]:
# Папка с леммами
lemmas_dir = os.path.join("..", "data", "lemmas")

# Получаем все файлы, начинающиеся с 'cleaned_'
files = glob.glob(os.path.join(lemmas_dir, "cleaned_*.csv"))

# Определяем гапаксы для каждой коллекции и их соотношение к общему числу лемм в каждом сборнике

In [ ]:
# Собираем все данные в один DataFrame
all_data = []
for file in files:
    # Читаем CSV без заголовка и задаём имя колонки "lemma"
    df = pd.read_csv(file, header=None, names=["lemma"])

    # Удаляем "cleaned_" из имени файла
    filename = os.path.basename(file).replace("cleaned_", "")

    # Добавляем колонку с названием сборника
    df["source"] = filename

    # Убираем дубликаты (на всякий случай)
    df = df.drop_duplicates(subset=["lemma", "source"])

    all_data.append(df)

# Объединяем все в один DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Подсчитываем, в скольких сборниках встречается каждая лемма
df_counts = df_all.groupby("lemma")["source"].nunique().reset_index()
df_counts.columns = ["lemma", "collection_count"]

# Присоединяем эту информацию обратно к основному DataFrame
df_all = df_all.merge(df_counts, on="lemma", how="left")

# Определяем уникальность леммы
df_all["is_unique"] = df_all["collection_count"] == 1

# Группируем по сборнику и считаем уникальные/неуникальные леммы
df_stats = df_all.groupby("source")["is_unique"].value_counts().unstack().fillna(0)

# Переименовываем столбцы
df_stats.columns = ["не уникальные леммы", "уникальные леммы"]

# Считаем общее количество лемм
df_stats["всего лемм"] = (
    df_stats["не уникальные леммы"] + df_stats["уникальные леммы"]
)

# Считаем долю уникальных лемм
df_stats["% уникальных лемм"] = (
    df_stats["уникальные леммы"] / df_stats["всего лемм"] * 100
).round(2)

# Просмотр результата
df_stats.head()


,не уникальные леммы,уникальные леммы,всего лемм,% уникальных лемм
source,,,,
alanus_de_insula_dist_summa.predicted_лем.csv,941,484,1425,33.96
anonymous_angelus.predicted_лем.csv,625,325,950,34.21
guido_ebroicensis_summa.predicted_лем.csv,70,5,75,6.67
mauritius_hibernicus_distinctiones.predicted_лем.csv,641,488,1129,43.22
nicolas_de_biard-distinctiones.predicted_лем.csv,555,311,866,35.91


In [8]:
df_stats.to_csv("../output/stats_tables/уникальные_леммы.csv")